In [1]:
import pandas as pd
import psycopg2

def fetch_query_records(query, columns):
    """
    Creates a connection to database, returns query from specified table.

    Input: query: a SQL query (string)

    Returns: response: cursos.fetchall() object in array form
    """
    DB_USER="citrics"
    DB_PASSWORD="BnDW2WupbFpgZSewsZm7"
    DB_NAME="postgres"
    DB_HOST="citricsads.cav8gkdxva9e.us-east-1.rds.amazonaws.com"

    # Creating Connection Object
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST)
    # Creating Cursor Object
    cursor = conn.cursor()
    # Fetch query
    query = query
    # Execute query
    cursor.execute(query)
    # Query results
    response = list(cursor.fetchall())
    # Closing Connection
    conn.close()

    return response

def fetch_query(query, columns):
    """
    Creates a connection to database, returns query from specified table
    as a list of dictionaries.

    Input: query: a SQL query (string)

    Returns: pairs: dataframe of cursor.fetchall() response in JSON pairs
    """
    
    # Fetch query
    response = fetch_query_records(query, columns)

    # Key-value pair names for df columns
    columns = columns
    # List of tuples to DF
    df = pd.DataFrame(response, columns=columns)
    # DF to dictionary
    pairs = df.to_json(orient='records')

    return pairs

# Goal: Compile `census`, `bls_jobs`, `walkability`, and `rental` into single, merged dataset
- Should align with front-end form where end user can enter preferences for:
    - Population
    - Average Rent
    - WalkScore
    - Primary Job Industry

### Start by creating df instances from each table in DB via SQL queries

In [2]:
### Rental Data

query = """
SELECT *
FROM rental
WHERE "month" = '2020-08-01'
"""

columns = ["month", "city", "state", "Studio", "onebr", "twobr", "threebr", "fourbr"]

rental = pd.read_json(fetch_query(query, columns))

rental = rental.drop("month", axis=1)

rental.columns = [x.lower() for x in rental.columns.to_list()]

print(rental.shape)
rental.head()

(444, 7)


,city,state,studio,onebr,twobr,threebr,fourbr
0,King of Prussia,PA,1038,1232,1488,1862,2072
1,Las Cruces,NM,514,599,726,1045,1280
2,North East,MD,909,1079,1303,1630,1814
3,St. Cloud,MN,578,712,883,1218,1543
4,Pasadena,CA,1350,1606,2064,2806,3093


In [3]:
### BLS Data

query = """
SELECT DISTINCT ON (j.city) j.*
FROM bls_jobs j
ORDER BY j.city, j.loc_quotient DESC
"""

columns = [
    "city",
    "state",
    "occ_title",
    "jobs_1000",
    "loc_quotient",
    "hourly_wage",
    "annual_wage"]

bls = pd.read_json(fetch_query(query, columns))

bls = bls.drop(["jobs_1000","loc_quotient"], axis=1)

bls = bls.loc[bls.annual_wage != 0]

print(bls.shape)
bls.head()

(362, 5)


,city,state,occ_title,hourly_wage,annual_wage
0,Abilene,TX,Wellhead Pumpers,30.14,62680
1,Aguadilla-Isabela,PR,Telemarketers,8.16,16980
2,Akron,OH,"Patternmakers, Metal and Plastic",17.27,35930
3,Albany,OR,Fallers,35.98,74850
4,Albany-Schenectady-Troy,NY,"Physical Scientists, All Other",46.86,97470


In [4]:
### WalkScores

query = """
SELECT *
FROM WALKABILITY
"""

columns = ["city", "walkscore"]

walkscores = pd.read_json(fetch_query(query, columns))

walkscores["state"] = [x[-2:] for x in walkscores.city.to_list()]
walkscores["city"] = [x.split(",")[0] for x in walkscores.city.to_list()]

walkscores = walkscores[["city","state","walkscore"]]

print(walkscores.shape)
walkscores.head()

(444, 3)


,city,state,walkscore
0,Houston,TX,86.28
1,Philadelphia,PA,97.70
2,Phoenix,AZ,71.76
3,San Antonio,TX,65.46
4,San Diego,CA,80.94


In [5]:
### Census Data

query = """
SELECT DISTINCT ON (j.city)
	j.city,
	j. "state",
	j.popestimate2019
FROM
	census j
ORDER BY
	j.city,
	j.popestimate2019 DESC
"""

# Re-factored query to reduce num of duplicates

columns = [
    "city",
    "state",
    "popestimate2019"]

census = pd.read_json(fetch_query(query, columns))

print(census.shape)
census.head()

(31802, 3)


,city,state,popestimate2019
0,Aastad township,MN,213
1,Abbeville city,LA,12038
2,Abbeville County,SC,24527
3,Abbeville town,MS,423
4,Abbotsford city,WI,2233


In [6]:
census.loc[census.city == "Baltimore city"]

# Problem with census data is the term 'city' after city names ...
# Also has several duplicates ...

,city,state,popestimate2019
5455,Baltimore city,MD,593490


In [7]:
census.city[1].replace(" city","")

'Abbeville'

In [8]:
census.city = [x.replace(" city","") for x in census.city.to_list()]

print(census.shape)
census.head()

(31802, 3)


,city,state,popestimate2019
0,Aastad township,MN,213
1,Abbeville,LA,12038
2,Abbeville County,SC,24527
3,Abbeville town,MS,423
4,Abbotsford,WI,2233


In [9]:
census.loc[(census.city == "Baltimore") & (census.state == "MD")]
# Now looks more appropriate

,city,state,popestimate2019
5455,Baltimore,MD,593490


# Explore Climate Data

In [11]:
PATH = "/Users/ekselan/Desktop/LAMBDA/LABS/Labs26-Citrics-DS-TeamA/data/climate/climate.csv"

climate = pd.read_csv(PATH)

climate.columns = [x.lower() for x in climate.columns.to_list()]


print(climate.shape)
climate.head()

(43191, 8)


,zip,city,state,latitude,longitude,timezone,daylight savings time flag,climate zone
0,67553,Liebenthal,KS,38.654948,-99.320620,CENTRAL STANDARD TIME,1,"Humid subtropical, no dry season"
1,85743,Tucson,AZ,32.335122,-111.148880,MOUNTAIN DAYLIGHT TIME,0,Subtropical steppe
2,75016,Irving,TX,32.767268,-96.777626,CENTRAL STANDARD TIME,1,"Humid subtropical, no dry season"
3,60401,Beecher,IL,41.350484,-87.624080,CENTRAL STANDARD TIME,1,"Humid continental, no dry season, hot summer"
4,80432,Como,CO,39.243440,-105.794310,MOUNTAIN DAYLIGHT TIME,1,"Subartic, severe winter, no dry season, cool s..."


In [20]:
climate.isnull().sum()

zip                            0
city                           0
state                          0
latitude                       0
longitude                      0
timezone                       0
daylight savings time flag     0
climate zone                  61
dtype: int64

In [21]:
climate.loc[climate["climate zone"].isnull()]

,zip,city,state,latitude,longitude,timezone,daylight savings time flag,climate zone
854,96847,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
1426,99591,Saint George Island,AK,56.605546,-169.559580,HAWAII-ALEUTIAN STANDARD TIME,1,NaN
1867,96808,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
2631,96857,Schofield Barracks,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
2938,96807,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
...,...,...,...,...,...,...,...,...
38904,96803,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
39322,96801,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
39546,96830,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN
39579,96844,Honolulu,HI,24.859832,-168.021815,HAWAII-ALEUTIAN STANDARD TIME,0,NaN


In [22]:
climate = climate.dropna()
print(climate.shape)
climate.head()

(43130, 8)


,zip,city,state,latitude,longitude,timezone,daylight savings time flag,climate zone
0,67553,Liebenthal,KS,38.654948,-99.320620,CENTRAL STANDARD TIME,1,"Humid subtropical, no dry season"
1,85743,Tucson,AZ,32.335122,-111.148880,MOUNTAIN DAYLIGHT TIME,0,Subtropical steppe
2,75016,Irving,TX,32.767268,-96.777626,CENTRAL STANDARD TIME,1,"Humid subtropical, no dry season"
3,60401,Beecher,IL,41.350484,-87.624080,CENTRAL STANDARD TIME,1,"Humid continental, no dry season, hot summer"
4,80432,Como,CO,39.243440,-105.794310,MOUNTAIN DAYLIGHT TIME,1,"Subartic, severe winter, no dry season, cool s..."


In [14]:
print("Number of Unqiue City Names in Climate Data:", climate.city.nunique())
print("Number of Unique climate zones in Climate Data:", climate["climate zone"].nunique())

Number of Unqiue City Names in Climate Data: 19053
Number of Unique climate zones in Climate Data: 21


In [ ]:
set(climate["climate zone"])

In [18]:
climate_map = {
'Humid continental, dry warm summer':'hot',
'Humid continental, no dry season, hot summer':'mild',
'Humid continental, no dry season, warm summer':'mild',
'Humid continental, severe dry winter, hot summer':'mild',
'Humid continental, severe dry winter, warm summer':'cold',
'Humid subtropical, no dry season':'hot',
'Marine west coast, cool summer':'cold',
'Marine west coast, warm summer':'mild',
'Mediterranean, hot summer':'hot',
'Mediterranean, warm summer':'mild',
'Mid-latitude desert':'mild',
'Mid-latitude steppe':'mild',
'Subartic, severe winter, no dry season, cool summer':'cold',
'Subartic, subalpine':'cold',
'Subtropical desert':'hot',
'Subtropical steppe':'mild',
'Tropical monsoon':'hot',
'Tropical rainforest':'hot',
'Tropical wet and dry or savanna':'hot',
"Tropical wet and dry or savanna ('summer' dry season)":'hot',
'Tundra':'cold'
}

In [19]:
climates = []

for row in climate["climate zone"]:
    for k,v in climate_map.items():
        if row == k:
            simple_climate = v
            climates.append(v)
            
len(climates)

43130

In [26]:
climate["simple_climate"] = climates

print(climate.shape)
climate.head()

(43130, 9)


,zip,city,state,latitude,longitude,timezone,daylight savings time flag,climate zone,simple_climate
0,67553,Liebenthal,KS,38.654948,-99.320620,CENTRAL STANDARD TIME,1,"Humid subtropical, no dry season",hot
1,85743,Tucson,AZ,32.335122,-111.148880,MOUNTAIN DAYLIGHT TIME,0,Subtropical steppe,mild
2,75016,Irving,TX,32.767268,-96.777626,CENTRAL STANDARD TIME,1,"Humid subtropical, no dry season",hot
3,60401,Beecher,IL,41.350484,-87.624080,CENTRAL STANDARD TIME,1,"Humid continental, no dry season, hot summer",mild
4,80432,Como,CO,39.243440,-105.794310,MOUNTAIN DAYLIGHT TIME,1,"Subartic, severe winter, no dry season, cool s...",cold


In [32]:
# Sanity Check the simple_climate

climate.loc[climate["simple_climate"] == "cold"].state.unique()

array(['CO', 'AK', 'MT', 'ID', 'UT', 'WY', 'WA', 'ND'], dtype=object)

In [33]:
climate.loc[climate["simple_climate"] == "mild"].state.unique()

array(['AZ', 'IL', 'KS', 'CO', 'MN', 'ID', 'MI', 'ND', 'IA', 'CA', 'OH',
       'WA', 'WI', 'NE', 'PA', 'NY', 'MT', 'IN', 'NH', 'CT', 'NC', 'NM',
       'TX', 'AK', 'UT', 'NJ', 'WV', 'MA', 'SD', 'VT', 'MD', 'MO', 'OR',
       'NV', 'ME', 'VA', 'RI', 'OK', 'WY', 'TN', 'HI', 'SC'], dtype=object)

In [34]:
climate.loc[climate["simple_climate"] == "hot"].state.unique()

array(['KS', 'TX', 'CA', 'AL', 'OH', 'TN', 'AR', 'OK', 'MO', 'CT', 'PA',
       'ID', 'WV', 'GA', 'FL', 'PR', 'NJ', 'NY', 'VA', 'NC', 'MD', 'LA',
       'KY', 'DE', 'DC', 'SC', 'IN', 'MS', 'IL', 'MA', 'VI', 'AZ', 'UT',
       'WA', 'OR', 'HI', 'RI', 'NV', 'CO', 'MT'], dtype=object)

In [51]:
climate.loc[(climate.city == "Las Cruces") & (climate.state == "NM")]

,zip,city,state,latitude,longitude,timezone,daylight savings time flag,climate zone,simple_climate
1908,88006,Las Cruces,NM,32.305193,-106.786259,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
9510,88001,Las Cruces,NM,32.300193,-106.760780,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
18748,88003,Las Cruces,NM,32.279944,-106.754100,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
19508,88012,Las Cruces,NM,32.417871,-106.697700,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
26493,88005,Las Cruces,NM,32.347752,-106.814630,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
32237,88085,Las Cruces,NM,32.336251,-106.792993,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
37639,88007,Las Cruces,NM,32.356091,-106.830111,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
37968,88011,Las Cruces,NM,32.312506,-106.703060,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild
39288,88004,Las Cruces,NM,32.204171,-106.930520,MOUNTAIN DAYLIGHT TIME,1,Mid-latitude desert,mild


In [59]:
# Multiple records for cities causing issue with merge - create subset

climate_sub = climate.sort_values(['city', 'zip'],
                        ascending=False).groupby(['city','state']).head(1)
climate_sub = climate_sub.reset_index()
climate_sub = climate_sub.drop("index", axis=1)
climate_sub = climate_sub.rename(columns={"climate zone":"climate_zone"})
climate_sub = climate_sub[["city","state","climate_zone","simple_climate"]]

print(climate_sub.shape)
climate_sub.head()

(30322, 4)


,city,state,climate_zone,simple_climate
0,Zwolle,LA,"Humid subtropical, no dry season",hot
1,Zwingle,IA,"Humid continental, no dry season, hot summer",mild
2,Zurich,MT,Mid-latitude steppe,mild
3,Zuni,NM,Mid-latitude steppe,mild
4,Zuni,VA,"Humid subtropical, no dry season",hot


# Inspect Data sets

In [35]:
print(walkscores.shape)
print(census.shape)
print(rental.shape)
print(bls.shape)
print(climate.shape)

(444, 3)
(31802, 3)
(444, 7)
(362, 5)
(43130, 9)


In [36]:
print("Walkscore states:", walkscores.state.nunique())
print("Census states:", census.state.nunique())
print("Rental states:", rental.state.nunique())
print("BLS states:", bls.state.nunique())
print("Climate states:", climate.state.nunique())

Walkscore states: 46
Census states: 51
Rental states: 46
BLS states: 90
Climate states: 53


In [47]:
codes = [
    'AL',
    'AK',
    'AZ',
    'AR',
    'CA',
    'CO', 
    'CT',
    'DE', 
    'DC',
    'FL',
    'GA',
    'HI',
    'ID',
    'IL',
    'IN',
    'IA',
    'KS',
    'KY',
    'LA',
    'ME',
    'MD',
    'MA',
    'MI',
    'MN',
    'MS',
    'MO',
    'MT',
    'NE',
    'NV',
    'NH',
    'NJ',
    'NM',
    'NY',
    'NC',
    'ND',
    'OH',
    'OK',
    'OR',
    'PA',
    'RI',
    'SC',
    'SD',
    'TN',
    'TX',
    'UT',
    'VT',
    'VA',
    'WA',
    'WV',
    'WI', 
    'WY'
]

len(codes)

51

In [58]:
for state in codes:
    if state not in walkscores.state.to_list():
        print(state)

HI
ME
SD
VT
WV


In [59]:
for state in codes:
    if state not in rental.state.to_list():
        print(state)
        
# The positive here is that the missing states are consistent
# between walkscores and rental data, but future iterations
# could strive to include all US states

HI
ME
SD
VT
WV


In [110]:
# Inspect BLS Data - deal with city and state names

print(bls.shape)
bls.head()

(362, 5)


,city,state,occ_title,hourly_wage,annual_wage
0,Abilene,TX,Wellhead Pumpers,30.14,62680
1,Aguadilla-Isabela,PR,Telemarketers,8.16,16980
2,Akron,OH,"Patternmakers, Metal and Plastic",17.27,35930
3,Albany,OR,Fallers,35.98,74850
4,Albany-Schenectady-Troy,NY,"Physical Scientists, All Other",46.86,97470


In [111]:
bls.loc[bls.city.str.contains("Dallas")]

# Can simply go with first city in city names for creation
# of static endpoint. BLS endpoint itself is more robust
# so in ex. below, 'Fort Worth' can still be found. On
# `static` endpoint however, it would not be found.

# Considering that this `static` endpoint is only intended for use with
# the advanced search feature, this may be a workable approach ...

,city,state,occ_title,hourly_wage,annual_wage
83,Dallas-Fort Worth-Arlington,TX,Semiconductor Processing Technicians,17.86,37150


In [37]:
# Make copy
bls_slim = bls.copy()

In [38]:
bls_slim["city"] = [x.split("-")[0].strip() for x in bls_slim.city.to_list()]
bls_slim["state"] = [x.split("-")[0].strip() for x in bls_slim.state.to_list()]

In [39]:
print(bls_slim.shape)
bls_slim.head()

# This should be able to merge better with m2 ...

(362, 5)


,city,state,occ_title,hourly_wage,annual_wage
0,Abilene,TX,Wellhead Pumpers,30.14,62680
1,Aguadilla,PR,Telemarketers,8.16,16980
2,Akron,OH,"Patternmakers, Metal and Plastic",17.27,35930
3,Albany,OR,Fallers,35.98,74850
4,Albany,NY,"Physical Scientists, All Other",46.86,97470


## Perform Merges - BLS Last Due to Naming Conventions

In [40]:
# Rental + Walkscores

m1 = rental.merge(walkscores, how="inner", on=["city","state"])

print(m1.shape)
m1.head()

(444, 8)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore
0,King of Prussia,PA,1038,1232,1488,1862,2072,53.64
1,Las Cruces,NM,514,599,726,1045,1280,56.08
2,North East,MD,909,1079,1303,1630,1814,41.96
3,St. Cloud,MN,578,712,883,1218,1543,64.32
4,Pasadena,CA,1350,1606,2064,2806,3093,90.48


In [41]:
print("Merge 1 Unique States:", m1.state.nunique())
print("Merge 1 Unique Cities:", m1.city.nunique())

Merge 1 Unique States: 46
Merge 1 Unique Cities: 429


In [42]:
# Rental + Walkscores + Census

m2 = m1.merge(census, how="inner", on=["city","state"])

print(m2.shape)
m2.head()

(365, 9)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,popestimate2019
0,Las Cruces,NM,514,599,726,1045,1280,56.08,103432
1,St. Cloud,MN,578,712,883,1218,1543,64.32,68462
2,Crystal Lake,IL,1042,1209,1422,1819,2165,60.68,39829
3,Clarksville,TN,523,631,810,1109,1241,45.90,158146
4,Englewood,CO,1003,1225,1551,2253,2610,65.86,34917


In [43]:
print("Merge 2 Unique States:", m2.state.nunique())
print("Merge 2 Unique Cities:", m2.city.nunique())

Merge 2 Unique States: 46
Merge 2 Unique Cities: 365


In [44]:
# Rental + Walkscores + Census + BLS

m3 = m2.merge(bls_slim, how="inner", on=["city","state"])

m3 = m3.rename(columns={"popestimate2019":"population"})

print(m3.shape)
m3.head()

(135, 12)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,population,occ_title,hourly_wage,annual_wage
0,Las Cruces,NM,514,599,726,1045,1280,56.08,103432,Anthropologists and Archeologists,28.75,59800
1,St. Cloud,MN,578,712,883,1218,1543,64.32,68462,Ophthalmic Laboratory Technicians,15.31,31850
2,Clarksville,TN,523,631,810,1109,1241,45.90,158146,"Insulation Workers, Floor, Ceiling, and Wall",9.40,19560
3,Tyler,TX,659,733,898,1187,1297,56.72,106985,Helpers--Extraction Workers,20.89,43450
4,Fresno,CA,803,851,1063,1508,1763,67.66,531576,"Farmworkers and Laborers, Crop, Nursery, and G...",12.36,25710


In [45]:
print("Merge 3 Unique States:", m3.state.nunique())
print("Merge 3 Unique Cities:", m3.city.nunique())

Merge 3 Unique States: 45
Merge 3 Unique Cities: 135


In [48]:
for state in codes:
    if state not in m3.state.to_list():
        print(state)
        
# Idaho now added to list of states not included ...

# Still able to retain 45 states though which is encouraging. Again,
# future iterations will likely involve a concerted effort to retain more states
# for each feature

HI
ID
ME
SD
VT
WV


In [49]:
# Do some spot checks to ensure 'major' cities are still
# represented in m3

m3.loc[m3.city == "Seattle"]

# After spot checking several "major" cities, I am satisfied
# with the retention, with the only concerning absence (out
# of about 20 experiments) being "Newark"


### UPDATE: "Seattle" is now a missing city as well due to not
# having wage data, but in consideration of the advanced search,
# it will be important for that data to be clean and void of
# missing information

,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,population,occ_title,hourly_wage,annual_wage


### `m4` - Add climate data

In [61]:
m4 = m3.merge(climate_sub, how="inner", on=["city","state"])

print(m4.shape)
m4.head()

(133, 14)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,population,occ_title,hourly_wage,annual_wage,climate_zone,simple_climate
0,Las Cruces,NM,514,599,726,1045,1280,56.08,103432,Anthropologists and Archeologists,28.75,59800,Mid-latitude desert,mild
1,Clarksville,TN,523,631,810,1109,1241,45.90,158146,"Insulation Workers, Floor, Ceiling, and Wall",9.40,19560,"Humid subtropical, no dry season",hot
2,Tyler,TX,659,733,898,1187,1297,56.72,106985,Helpers--Extraction Workers,20.89,43450,"Humid subtropical, no dry season",hot
3,Fresno,CA,803,851,1063,1508,1763,67.66,531576,"Farmworkers and Laborers, Crop, Nursery, and G...",12.36,25710,Mid-latitude steppe,mild
4,Milwaukee,WI,604,734,910,1146,1287,83.78,590157,Computer Numerically Controlled Tool Operators,22.36,46500,"Humid continental, no dry season, warm summer",mild


In [62]:
m4.state.nunique()

45

In [63]:
m4.loc[m4.city == "Atlanta"]

,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,population,occ_title,hourly_wage,annual_wage,climate_zone,simple_climate
96,Atlanta,GA,979,1027,1187,1558,1914,77.82,506811,"Miscellaneous Entertainers and Performers, Spo...",13.61,28308,"Humid subtropical, no dry season",hot


In [64]:
### Export m3

PATH = "/Users/ekselan/Desktop/LAMBDA/LABS/Labs26-Citrics-DS-TeamA/data/static/static.csv"

m4.to_csv(PATH, index=False)